In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Parallel ADK Agent Demo

> **Note**: This notebook is for reference and educational purposes only. Not intended for production use.  
> **Questions?** mateuswagner at google.com

This notebook demonstrates how to build a **parallel agent** using Google's Agent Development Kit (ADK).

## What is a ParallelAgent?

A **ParallelAgent** is a workflow agent that executes multiple sub-agents **concurrently**, enabling:
- Faster execution by running independent tasks simultaneously
- Concurrent data retrieval from multiple sources
- Parallel processing of independent operations

**Tech Stack**: ADK with Gemini 3

---

## Get started

In [ ]:
## Installation
# Python 3.10+ recommended
# Virtual environment (venv) recommended for isolation
# Google Cloud SDK initialized: `gcloud init`

%pip install --upgrade --quiet 'google-adk' nbformat

# Restart your Jupyter kernel !


## Import libraries

In [ ]:
# Standard library imports
import asyncio
import json
import os
import warnings
from typing import Any

# Third-party imports
from IPython.display import Markdown, display

# Google ADK imports
from google.adk.agents import Agent, ParallelAgent
from google.adk.events import Event
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

### CONFIGURATION SETTINGS

In [ ]:
# Google Cloud Project Configuration
# CHANGE THIS: Set your Google Cloud project ID
PROJECT_ID = "matt-demos" # CHANGE IT!

# Fallback to environment variable if not set
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

# Default region for Vertex AI resources
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

# Set environment variables
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

## Build ADK agent

Build your application using ADK, including the Gemini model and custom tools that you define.

---


### Set agent tools

To start, set the tools that a customer support agent needs to do their job.

In [ ]:
def get_product_details(product_name: str):
    """Gathers basic details about a product."""
    details = {
        "smartphone": "A cutting-edge smartphone with advanced camera features and lightning-fast processing.",
        "usb charger": "A super fast and light usb charger",
        "shoes": "High-performance running shoes designed for comfort, support, and speed.",
        "headphones": "Wireless headphones with advanced noise cancellation technology for immersive audio.",
        "speaker": "A voice-controlled smart speaker that plays music, sets alarms, and controls smart home devices.",
    }
    return details.get(product_name, "Product details not found.")


def get_product_price(product_name: str):
    """Gathers price about a product."""
    details = {
        "smartphone": 500,
        "usb charger": 10,
        "shoes": 100,
        "headphones": 50,
        "speaker": 80,
    }
    return details.get(product_name, "Product price not found.")

### Set Agent the model

Configure the Gemini model for your ADK agent. This notebook uses **`gemini-3-pro-preview`** for fast, cost-effective function calling.

See the [Gemini model documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models) for detailed performance benchmarks and pricing.

In [ ]:
model = "gemini-3-pro-preview" # review it

### Assemble the ParallelAgent

A **ParallelAgent** executes multiple sub-agents concurrently, which is ideal when:
- Tasks can be performed independently without dependencies
- You need to retrieve data from multiple sources simultaneously
- You want to speed up workflows by running agents in parallel

**Key characteristics:**
- Not powered by an LLM (deterministic execution)
- Sub-agents run independently with no automatic state sharing
- All sub-agents start running at approximately the same time

In [ ]:
async def run_parallel_agent_demo():
    """
    Demonstrates ParallelAgent by running multiple product research agents concurrently.
    A coordinator agent waits for all sub-agents and consolidates results.
    """
    # Session identifiers for tracking agent interactions
    app_name = "parallel_product_research_app"
    user_id = "user_one"
    session_id = "session_one"
    
    # Create individual agents for different product research tasks
    price_agent = Agent(
        name="PriceAgent",
        model=model,
        description="Agent that researches product prices",
        instruction="Get the price for the smartphone product using the get_product_price tool. Report: 'Smartphone price: $[PRICE]'",
        tools=[get_product_price],
    )
    
    details_agent = Agent(
        name="DetailsAgent",
        model=model,
        description="Agent that researches product details",
        instruction="Get the product details for headphones using the get_product_details tool. Report: 'Headphones: [FULL DESCRIPTION]'",
        tools=[get_product_details],
    )
    
    # Create another agent for a different product
    shoes_agent = Agent(
        name="ShoesAgent",
        model=model,
        description="Agent that researches shoes",
        instruction="Get the product details for shoes using the get_product_details tool. Report: 'Shoes: [FULL DESCRIPTION]'",
        tools=[get_product_details],
    )
    
    # Create ParallelAgent that will run all sub-agents concurrently
    parallel_agent = ParallelAgent(
        name="ParallelProductResearchAgent",
        description="Coordinates parallel product research across multiple agents",
        sub_agents=[price_agent, details_agent, shoes_agent],
    )
    
    # Create coordinator agent that waits for all results and consolidates
    coordinator_agent = Agent(
        name="CoordinatorAgent",
        model=model,
        description="Waits for all parallel research agents and consolidates their findings",
        instruction="""You are a research coordinator managing three research teams in parallel.

CRITICAL TASK:
1. Delegate work to the ParallelProductResearchAgent (which runs 3 teams concurrently)
2. WAIT for ALL three teams to complete their research:
   - PriceAgent (smartphone pricing)
   - DetailsAgent (headphones details)
   - ShoesAgent (shoes details)
3. Once you receive results from ALL THREE teams, create a comprehensive final report

FINAL REPORT FORMAT:
Create a detailed summary with sections for each product:

## Product Research Summary

### Smartphone
- **Price**: [include price from PriceAgent]
- **Details**: [any additional info if available]

### Headphones
- **Description**: [full description from DetailsAgent]
- **Price**: [if available]

### Shoes
- **Description**: [full description from ShoesAgent]
- **Price**: [if available]

IMPORTANT: 
- Include ALL information received from ALL agents
- Do not respond until you have gathered results from all three teams
- Be comprehensive and include every detail
- If a team did not provide certain information, state "Not available" """,
        sub_agents=[parallel_agent],
    )
    
    # Initialize in-memory session storage
    session_service = InMemorySessionService()
    await session_service.create_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )
    
    # Create runner to execute the coordinator agent
    runner = Runner(
        agent=coordinator_agent, 
        app_name=app_name, 
        session_service=session_service,
    )
    
    # Run the coordinator - it will execute parallel agents and consolidate results
    query = "Coordinate the product research teams and provide me a comprehensive final report with all findings"
    content = types.Content(role="user", parts=[types.Part(text=query)])
    
    print("Starting parallel agent execution with coordinator...")
    print("Running 3 agents concurrently: PriceAgent, DetailsAgent, and ShoesAgent")
    print("Coordinator will wait for all agents and consolidate results\n")
    
    # Collect all events
    events = []
    async for event in runner.run_async(
        user_id=user_id, 
        session_id=session_id, 
        new_message=content
    ):
        events.append(event)
    
    return events

---

## Parallel Agent Demo with Coordinator

This demo runs **ParallelAgent** with a **Coordinator Agent**:

1. **Parallel Execution**: 3 agents run concurrently
   - **PriceAgent**: Fetches smartphone price
   - **DetailsAgent**: Fetches headphones details  
   - **ShoesAgent**: Fetches shoes details

2. **Coordinator Display**: Shows ALL information from all sub-agents without filtering or summarizing

The coordinator acts as a pass-through, displaying complete results from each parallel agent.

In [ ]:
# Test the Parallel Agent with Coordinator

events = await run_parallel_agent_demo()

# Extract and display the final consolidated answer from the Coordinator
print("\n" + "="*80)
print("FINAL CONSOLIDATED ANSWER FROM COORDINATOR")
print("="*80 + "\n")

coordinator_final_answer = None
for event in reversed(events):  # Start from the end to find coordinator's response
    if (hasattr(event, 'author') and event.author == 'CoordinatorAgent' and
        hasattr(event, 'content') and event.content and
        hasattr(event.content, 'parts')):
        for part in event.content.parts:
            if hasattr(part, 'text') and part.text:
                coordinator_final_answer = part.text
                break
        if coordinator_final_answer:
            break

if coordinator_final_answer:
    display(Markdown(coordinator_final_answer))
else:
    print("WARNING: No final answer from Coordinator found")
    print("\nShowing all events for debugging:\n")
    
    for i, event in enumerate(events, 1):
        print(f"\n{'='*80}")
        print(f"EVENT {i}/{len(events)} - Author: {getattr(event, 'author', 'Unknown')}")
        print(f"{'='*80}")
        
        if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts'):
            print(f"  Role: {getattr(event.content, 'role', 'N/A')}")
            for j, part in enumerate(event.content.parts, 1):
                if hasattr(part, 'text') and part.text:
                    print(f"    Text: {part.text[:300]}...")
                if hasattr(part, 'function_call') and part.function_call:
                    print(f"    Function Call: {part.function_call.name}")
                if hasattr(part, 'function_response') and part.function_response:
                    print(f"    Function Response: {part.function_response.name}")

In [ ]:
# Dump all events:

print("\n" + "="*80)
print(f"TOTAL EVENTS COLLECTED: {len(events)}")
print("="*80 + "\n")

for i, event in enumerate(events, 1):
    print(f"\n{'='*80}")
    print(f"EVENT {i}/{len(events)}")
    print(f"{'='*80}")
    
    # Show event type
    print(f"Event Type: {type(event).__name__}")
    
    # Show event attributes
    if hasattr(event, '__dict__'):
        for key, value in event.__dict__.items():
            print(f"\n{key}:")
            if key == 'content' and hasattr(value, 'parts'):
                print(f"  Role: {getattr(value, 'role', 'N/A')}")
                print(f"  Parts ({len(value.parts)}):")
                for j, part in enumerate(value.parts, 1):
                    print(f"    Part {j}:")
                    if hasattr(part, 'text') and part.text is not None:
                        text = str(part.text)
                        print(f"      Text: {text[:200]}..." if len(text) > 200 else f"      Text: {text}")
                    if hasattr(part, 'function_call') and part.function_call is not None:
                        print(f"      Function Call: {part.function_call.name}")
                        print(f"      Args: {dict(part.function_call.args)}")
                    if hasattr(part, 'function_response') and part.function_response is not None:
                        print(f"      Function Response: {part.function_response.name}")
                        print(f"      Response: {part.function_response.response}")
            else:
                # Truncate long values
                str_value = str(value)
                if len(str_value) > 200:
                    print(f"  {str_value[:200]}...")
                else:
                    print(f"  {str_value}")

In [ ]:
# Extract and print the latest answer from events

print("\n" + "="*80)
print("LATEST ANSWER FROM EVENTS")
print("="*80 + "\n")

# Find the last event with text content from a model
latest_answer = None
latest_author = None

for event in reversed(events):
    if hasattr(event, 'content') and hasattr(event.content, 'parts'):
        for part in event.content.parts:
            if hasattr(part, 'text') and part.text:
                latest_answer = part.text
                latest_author = getattr(event, 'author', 'Unknown')
                break
        if latest_answer:
            break

if latest_answer:
    print(f"Author: {latest_author}")
    print(f"\n{latest_answer}")
else:
    print("No text response found in events.")

print("\n" + "="*80)

---

# Thank you